In [1]:
%matplotlib widget

import os
import pathlib
import pprint

from Visualizer.DataSet import DataSet
from Visualizer.Monitoring.Persistence.ProfilerSession import ProfilerSession
from Visualizer.Plotting.Plot import Plot

# [Analyzer] Single Profiler Session

This notebook can visualize a specific profiler session.

## Configuration

Configure the project settings below.
The default configuration should be okay.
Make sure that you have run at least one profiler session with the EnergyManager and have generated a database, otherwise this notebook will fail.

In [2]:
source_directory = pathlib.Path().parent.absolute()
project_directory = f"{source_directory}/.."
root_directory = f"{project_directory}/.."
resources_directory = f"{root_directory}/Resources"
database = f"{resources_directory}/Database.sqlite"

# Load all profiler sessions
complete_data_set = DataSet(ProfilerSession.load_all(database))

### Profiler Sessions Overview

Below all profiler sessions are shown.

In [3]:
# Visualize the profiler session data
overview_plot = complete_data_set.table_plot
overview_plot.maximum_rows = 500
overview_plot.plot()

,ID,Label,Profile
0,1,JacobiNoMPIProfiler,"{'Ni': 512, 'Nj': 512, 'file': '/applications/jacobi/small', 'iterations': 10000, 'kernel': 1}"
1,2,JacobiNoMPIProfiler,"{'Ni': 512, 'Nj': 512, 'file': '/applications/jacobi/small', 'iterations': 10000, 'kernel': 2}"
2,3,JacobiNoMPIProfiler (EnergyMonitor MinMax),"{'Ni': 512, 'Nj': 512, 'file': '/applications/jacobi/small', 'iterations': 10000, 'kernel': 1}"
3,4,JacobiNoMPIProfiler (EnergyMonitor MinMax),"{'Ni': 512, 'Nj': 512, 'file': '/applications/jacobi/small', 'iterations': 10000, 'kernel': 2}"
4,5,JacobiNoMPIProfiler (EnergyMonitor System),"{'Ni': 512, 'Nj': 512, 'file': '/applications/jacobi/small', 'iterations': 10000, 'kernel': 1}"
5,6,JacobiNoMPIProfiler (EnergyMonitor System),"{'Ni': 512, 'Nj': 512, 'file': '/applications/jacobi/small', 'iterations': 10000, 'kernel': 2}"
6,7,JacobiNoMPIProfiler,"{'Ni': 512, 'Nj': 512, 'file': '/applications/jacobi/small', 'iterations': 10000, 'kernel': 1}"
7,8,JacobiNoMPIProfiler,"{'Ni': 512, 'Nj': 512, 'file': '/applications/jacobi/small', 'iterations': 10000, 'kernel': 2}"
8,9,StreamclusterProfiler,"{'chunksize': 65536, 'clustersize': 1000, 'd': 256, 'infile': 'none', 'iterations': 10, 'k1': 10, 'k2': 20, 'n': 65536, 'nproc': 1, 'outfile': 'output.txt'}"
9,10,StreamclusterProfiler (EnergyMonitor MinMax),"{'chunksize': 65536, 'clustersize': 1000, 'd': 256, 'infile': 'none', 'iterations': 10, 'k1': 10, 'k2': 20, 'n': 65536, 'nproc': 1, 'outfile': 'output.txt'}"


## Analysis

Define the ID of the test you wish to visualize below.

In [20]:
#profiler_session_id = complete_data_set.data[-1].id
profiler_session_id = 3
data_set = DataSet([profiler_session for profiler_session in complete_data_set.data if profiler_session.id == profiler_session_id])

### Summary

These are the specifications of the system with which the profiler session was performed.

In [21]:
pprint.pprint(data_set.data[0].summary)

{'GPUs': [{'Brand': 'GeForce',
           'Compute Capability Major Version': 5,
           'Compute Capability Minor Version': 2,
           'Default Auto Boosted Clocks Enabled': None,
           'Default Power Limit (W)': '130.0',
           'Memory Size (B)': '2.1 GB',
           'Name': 'GeForce GTX 960',
           'PCIe Link Width (B)': '16 Bytes',
           'Supported Core Clock Rates (Hz)': [''],
           'Supported Memory Clock Rates (Hz)': ['']}],
 'Label': 'StreamclusterProfiler'}


### Profile

This is the profile with which the profiler session was performed.

In [22]:
pprint.pprint(data_set.data[0].profile)

{'chunksize': 65536,
 'clustersize': 1000,
 'd': 256,
 'infile': 'none',
 'iterations': 10,
 'k1': 10,
 'k2': 20,
 'n': 65536,
 'nproc': 1,
 'outfile': 'output.txt'}


The graph below shows the clock rate over time.

In [23]:
clock_rate_timeseries_plot = data_set.data[0].clock_rate_timeseries_plot(plot_limits=False, use_ear=False)
clock_rate_timeseries_plot.plot_series = { key: clock_rate_timeseries_plot.plot_series[key] for key in clock_rate_timeseries_plot.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0")}
clock_rate_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Correlations

The graphs below shows the Pearson correlation heatmap for each performance variable.

In [24]:
#data_set.data[0].monitor_data_correlations_plot.plot()

### Energy Consumption

The graphs below shows the total energy consumption over time.

In [25]:
#### # Plot in Joules
energy_consumption_plot_j = data_set.data[0].energy_consumption_timeseries_plot(use_ear=False)
energy_consumption_plot_j.plot_series = { key: energy_consumption_plot_j.plot_series[key] for key in energy_consumption_plot_j.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
energy_consumption_plot_j.plot()

# Plot in Watt Hours
# energy_consumption_plot_wh = data_set.data[0].energy_consumption_timeseries_plot(unit_string="Wh", modifier=Plot.j_to_wh, use_ear=False)
# energy_consumption_plot_wh.plot_series = { key: energy_consumption_plot_wh.plot_series[key] for key in energy_consumption_plot_wh.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
# energy_consumption_plot_wh.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Events

The graph below shows the events over time.

In [26]:
# for events_plot in data_set.data[0].events_event_plot()[:1]:
#     events_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Fan Speed

The graphs below shows the fan speed over time.

In [27]:
data_set.data[0].fan_speed_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Instructions Per Cycle

The graph below shows the utilization rates over time.

In [28]:
data_set.data[0].instructions_per_cycle_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

No handles with labels found to put in legend.


### Memory Consumption

The graphs below shows the memory consumption over time.

In [29]:
data_set.data[0].memory_consumption_timeseries_plot(plot_sizes=False).plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

JacobiNoMPIProfiler### Power Consumption

The graph below shows the power consumption over time.

In [30]:
power_consumption_timeseries_plot = data_set.data[0].power_consumption_timeseries_plot(plot_limits=False, use_ear=False)
#power_consumption_timeseries_plot.plot_series = { key: power_consumption_timeseries_plot.plot_series[key] for key in power_consumption_timeseries_plot.plot_series if (key.startswith("CPU 1") or key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
power_consumption_timeseries_plot.plot_series = { key: power_consumption_timeseries_plot.plot_series[key] for key in power_consumption_timeseries_plot.plot_series if (key.startswith("EAR") or key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
power_consumption_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Processes

The graphs below shows the amount of processes over time.

In [31]:
data_set.data[0].processes_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### States

The graph below shows the states of the EnergyMonitor over time.

In [32]:
data_set.data[0].states_event_plot().plot()

/EnergyManager/Visualizer/Source/Visualizer/Plotting/FigurePlot.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self.figure = pyplot.figure(figsize=(15, 5))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [17]:
total = 0
idle = 0
gpu_idle = 0
cpu_idle = 0
busy = 0
cpu_busy_wait = 0

for (state, begin, end) in data_set.data[0].states:
    diff = (end - begin).total_seconds()
    total += diff
    if (state == "IDLE"):
        idle += diff
    elif (state == "GPU_IDLE"):
        gpu_idle += diff
    elif (state == "CPU_IDLE"):
        cpu_idle += diff
    elif (state == "BUSY"):
        busy += diff
    elif (state == "CPU_BUSY_WAIT"):
        cpu_busy_wait += diff
    
n = 1
print("IDLE:", round(idle/total*100, n), "%\nCPU_IDLE:", round(cpu_idle/total*100, n), "%\nGPU_IDLE:", round(gpu_idle/total*100, n), "%\nCPU_BUSY_WAIT:", round(cpu_busy_wait/total*100, n), "%\nBUSY:", round(busy/total*100, n), "%")

IDLE: 0.3 %
CPU_IDLE: 0.0 %
GPU_IDLE: 0.9 %
CPU_BUSY_WAIT: 96.8 %
BUSY: 1.9 %


### Temperature

The graphs below shows the temperature over time.

In [33]:
temperature_timeseries_plot = data_set.data[0].temperature_timeseries_plot
temperature_timeseries_plot.plot_series = { key: temperature_timeseries_plot.plot_series[key] for key in temperature_timeseries_plot.plot_series if (key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
temperature_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Timespan

The graphs below shows the time spent on different types of operations over time.

In [34]:
timespan_timeseries_plot = data_set.data[0].timespan_timeseries_plot(use_ear=False)
timespan_timeseries_plot.plot_series = { key: timespan_timeseries_plot.plot_series[key] for key in timespan_timeseries_plot.plot_series if (key.startswith("Runtime") or key.startswith("EAR") or key.startswith("CPU 0 Core 0"))}
timespan_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### Utilization Rates

The graph below shows the utilization rates over time.

In [35]:
utilization_rate_timeseries_plot = data_set.data[0].utilization_rate_timeseries_plot
utilization_rate_timeseries_plot.plot_series = { key: utilization_rate_timeseries_plot.plot_series[key] for key in utilization_rate_timeseries_plot.plot_series if (key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
# utilization_rate_timeseries_plot.plot_series = { key: utilization_rate_timeseries_plot.plot_series[key] for key in utilization_rate_timeseries_plot.plot_series if (key.startswith("CPU 0") and (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 2")) or (not key.startswith("CPU 0 Core") or key.startswith("CPU 0 Core 0"))) or key.startswith("GPU 0") or key.startswith("Node")}
utilization_rate_timeseries_plot.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# 